In [ ]:
%load_ext autoreload
%autoreload 2

# First, import dependencies.
import numpy as np
import time, os, sys
from cellpose_omni import models, core

# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you are only processing a few images.
use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

# for plotting 
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

In [ ]:
files = ['/home/kcutler/DataDrive/DeLTA/delta/assets/trainingsets/2D/training/segmentation_set/img/Sample000005.png',
         '/home/kcutler/DataDrive/DeLTA/tests/data/movie_2D_tif/Position01Channel01Frame000008.tif']
files = files+['/home/kcutler/DataDrive/delta/delta/assets/trainingsets/2D/training/segmentation_set/img/Sample000033.png']

In [ ]:
from cellpose_omni import io, transforms 
imgs = [io.imread(f) for f in files]

# print some infor about the images 
for i in imgs:
    print(i.shape)
nimg = len(imgs)
print(nimg)

plt.figure(figsize=[4]*2) # initialize figure
for k in range(len(imgs)):
    img = transforms.move_min_dim(imgs[k]) # move the channel dimension last
    if len(img.shape)>2:
        # imgs[k] = img[:,:,1]
        imgs[k] = np.mean(img,axis=-1)
        
    imgs[k] = transforms.normalize99(imgs[k],omni=True)
    plt.subplot(1,len(files),k+1)
    plt.imshow(imgs[k],cmap='gray')
    plt.axis('off')

In [ ]:
model_name = 'bact_omni'
modeldir = '/home/kcutler/DataDrive/omnipose_train/registered/models/cellpose_residual_on_style_on_concatenation_off_omni_registered_2022_01_23_16_11_17.784983_epoch_1951'
model = [models.CellposeModel(gpu=use_GPU, model_type='bact_omni'),
         models.CellposeModel(gpu=use_GPU, pretrained_model=modeldir)]

In [ ]:
resample = True
verbose = False
cluster = True #use clustering algorithm for Omnipose 
chans = [0,0]
imglist = imgs[:]
nimg = len(imgs)

names = ['Omnipose','Omnipose_bit_loss']
omni = True
nmodel = len(names)
masks,flows,styles,d = [[]]*nmodel,[[]]*nmodel,[[]]*nmodel,[[]]*nmodel

for k in range(nmodel):
    masks[k], flows[k], styles[k] = model[k].eval(imglist,channels=chans,rescale=None,mask_threshold=-1,flow_threshold=0,
                                                  omni=True,cluster=cluster,resample=resample,tile=False,transparency=True)


In [ ]:
from cellpose_omni import plot
import omnipose

for j in range(nmodel):
    for k in range(nimg):
        print(names[j],model[j].pretrained_model)
        maski = masks[j][k]
        flowi = flows[j][k][0]
        fig = plt.figure(figsize=(12,5))
        # im = transforms.move_min_dim(imgs[i])
        # print(im.shape)
        plot.show_segmentation(fig, imgs[k], maski, flowi, channels=chans, omni=True, bg_color=0)
        plt.tight_layout()
        plt.show()